In [12]:
import instructor
from pydantic import BaseModel, Field
from typing import List
from openai import OpenAI

class PlayerResponse(BaseModel):
    player_id: int = Field(description="ID of the player")
    report_summary: str = Field(name="report_summary", description="Summary of the reports that have the same player id")
    

class ListPlayerResponse(BaseModel):
    list: List[PlayerResponse]

client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama"
    ),
    mode=instructor.Mode.JSON
)    


In [13]:
resp = client.chat.completions.create(
    model="mistral",
    messages=[{
        "role": "user",
        "content": "Player-ID: 1337, Report: Played as an attacking winger and used his speed and technique excellently. His dribbling led to several dangerous situations, although he again showed weaknesses defensively. He set up a goal with a precise cross and scored one himself after a solo run. His creativity and game vision were at a high level. At 23 years old, he still has a lot of potential to rise. \n\n Player-ID: 888, Report:  As a defensive midfielder, he again showed a disciplined and strong performance. Won many important duels in midfield and effectively stopped opponent attacks.His passing game was precise and he showed excellent vision. Offensively, he set no accents but focused on ball distribution. Showed weaknesses in fast counterattacks by the opponent. At 27 years old, he is at the peak of his footballing age and a reliable force in midfield.",
    }],
    response_model=ListPlayerResponse,
)

print(resp.model_dump_json(indent=2))

{
  "list": [
    {
      "player_id": 1337,
      "report_summary": "Played as an attacking winger and used his speed and technique excellently. His dribbling led to several dangerous situations, although he again showed weaknesses defensively. He set up a goal with a precise cross and scored one himself after a solo run. His creativity and game vision were at a high level. At 23 years old, he still has a lot of potential to rise."
    },
    {
      "player_id": 888,
      "report_summary": "As a defensive midfielder, he again showed a disciplined and strong performance. Won many important duels in midfield and effectively stopped opponent attacks. His passing game was precise and he showed excellent vision. Offensively, he set no accents but focused on ball distribution. Showed weaknesses in fast counterattacks by the opponent. At 27 years old, he is at the peak of his footballing age and a reliable force in midfield."
    }
  ]
}
